In [1]:
import os
import sys
import inspect
from dotenv import load_dotenv

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# to import backend module
currentdir = os.path.dirname(os.path.abspath(globals()['_dh'][0]))
if currentdir not in sys.path:
    sys.path.insert(0, currentdir) 

In [4]:
# env variables
load_dotenv('../.env.local')

True

In [19]:
from backend.ai import *
from backend.ai import clean
from backend.ast import *

In [6]:
data_head = """
UNIQUE_ID	PWPNT_LAT	PWPNT_LON	NAME	GADM_ISO	GADM_NAME	AREA	B75	B90	B00	B15	P75	P90	P00	P15	CLASS	Permalink									
4448	34.52843	69.14994	Kabul	AFG	Afghanistan	321	46.56713217	84.6349568	97.09434273	110.5077968	1111315.886	1347143.46	2426451.543	4370944.342	UC	https://ghsl.jrc.ec.europa.eu/visualisation.php#lnlt=@34.52843,69.14994,10z&v=201&ln=0&gr=ds&lv=000100000001000111&lo=aaa3aaaaaaa5aaaaaa&pg=V									
758	34.35273	62.21094	Herat	AFG	Afghanistan	58	4.962034005	8.537432097	11.37458099	13.31668304	147310.5829	209034.7942	410757.4447	840022.8842	UC	https://ghsl.jrc.ec.europa.eu/visualisation.php#lnlt=@34.35273,62.21094,10z&v=201&ln=0&gr=ds&lv=000100000001000111&lo=aaa3aaaaaaa5aaaaaa&pg=V									
"""

In [7]:
data_description = """
GHS Urban Centre Database 2015: national population and built-up area dynamics, v1.0, Afghanistan
COLUMN	SHORT DESC
UNIQUE_ID	Unique ID of Spatial entitity (Urban Centre; Dense Urban Cluster; Semi-Dense Urban Cluster)
PWPNT_LAT	Population Weighted Centroid Latitude (WGS 84)
PWPNT_LON	Population Weighted Centroid Longitude (WGS 84)
NAME	Spatial entity name (visualisation purpose)
GADM_ISO	GADM2.8 country: ISO 3
GADM_NAME	GADM2.8 country: name
AREA	Total area of the urban centre (km sq)
B75	Total built-up area in 1975 within the urban centre of 2015
B90	Total built-up area in 1990 within the urban centre of 2015
B00	Total built-up area in 2000 within the urban centre of 2015
B15	Total built-up area in 2015 within the urban centre of 2015
P75	Total population in 1975 within the urban centre of 2015
P90	Total population in 1990 within the urban centre of 2015
P00	Total population in 2000 within the urban centre of 2015
P15	Total population in 2015 within the urban centre of 2015
CLASS	Specifies the class of the spatial entity  (UC: Urban Centre; DUC: Dense Urban Cluster; SDUC: Semi-Dense Urban Cluster)
PLINK	Permalink to the interactive visualisation on the GHSL website https://ghsl.jrc.ec.europa.eu/visualisation.php
"""

In [15]:
result = import_code(data_head, data_description, model_name='gpt-4')

-- langchain-mod --
{'prompt_tokens': 977, 'completion_tokens': 510, 'total_tokens': 1487}
-- langchain-mod --


In [17]:
result

data = spark.read.csv("data.txt", header=True, inferSchema=True, sep="\t")

# create nodes data frame
nodes = data.select("UNIQUE_ID", "NAME", "PWPNT_LAT", "PWPNT_LON", "GADM_ISO", "GADM_NAME", "AREA", "CLASS") \
    .withColumnRenamed("UNIQUE_ID", "id") \
    .withColumnRenamed("NAME", "node_type") \
    .withColumnRenamed("PWPNT_LAT", "latitude") \
    .withColumnRenamed("PWPNT_LON", "longitude") \
    .withColumnRenamed("GADM_ISO", "country_iso") \
    .withColumnRenamed("GADM_NAME", "country_name") \
    .withColumnRenamed("AREA", "area") \
    .withColumnRenamed("CLASS", "class")

# create edges data frame
edges = data.select("UNIQUE_ID", "B75", "B90", "B00", "B15", "P75", "P90", "P00", "P15") \
    .withColumnRenamed("UNIQUE_ID", "source_id")

# create target nodes for edges
targets = ["B75", "B90", "B00", "B15", "P75", "P90", "P00", "P15"]
for target in targets:
    edges = edges.withColumn(target, edges["source_id"])

# rename columns for edges
edges = edges.withColumnRenamed("

In [20]:
cleaned = clean(result.result)
cleaned

'data = spark.read.csv("data.txt", header=True, inferSchema=True, sep="\\t")\n\n# create nodes data frame\nnodes = data.select("UNIQUE_ID", "NAME", "PWPNT_LAT", "PWPNT_LON", "GADM_ISO", "GADM_NAME", "AREA", "CLASS") \\\n    .withColumnRenamed("UNIQUE_ID", "id") \\\n    .withColumnRenamed("NAME", "node_type") \\\n    .withColumnRenamed("PWPNT_LAT", "latitude") \\\n    .withColumnRenamed("PWPNT_LON", "longitude") \\\n    .withColumnRenamed("GADM_ISO", "country_iso") \\\n    .withColumnRenamed("GADM_NAME", "country_name") \\\n    .withColumnRenamed("AREA", "area") \\\n    .withColumnRenamed("CLASS", "class")\n\n# create edges data frame\nedges = data.select("UNIQUE_ID", "B75", "B90", "B00", "B15", "P75", "P90", "P00", "P15") \\\n    .withColumnRenamed("UNIQUE_ID", "source_id")\n\n# create target nodes for edges\ntargets = ["B75", "B90", "B00", "B15", "P75", "P90", "P00", "P15"]\nfor target in targets:\n    edges = edges.withColumn(target, edges["source_id"])\n\n# rename columns for edges\

In [21]:
import ast

In [22]:
a = ast.parse(cleaned)

In [27]:
assign = a.body[0]

In [31]:
assign.value.func.value

# from agent_example.py

GPT4

```python
from brainshare import load_node, load_edge

def load_data(file_contents):
    lines = file_contents.split("\n")
    header = lines.pop(0).split("\t")

    for line in lines:
        if not line:
            continue
        data = line.split("\t")
        data_dict = {header[i]: data[i] for i in range(len(header))}

        city_data = {
            "name": data_dict["NAME"],
            "population": int(float(data_dict["P15"])),
            "latitude": float(data_dict["PWPNT_LAT"]),
            "longitude": float(data_dict["PWPNT_LON"]),
        }
        city_id = load_node("city", city_data)

        country_data = {
            "name": data_dict["GADM_NAME"],
            "iso": data_dict["GADM_ISO"],
        }
        country_id = load_node("country", country_data)

        load_edge(city_id, country_id)

load_data(file_contents)
```
--
AiResult: used 5952 token, cost $0.1886, took 74.9 seconds

GPT3

```python
from brainshare import *

# Define the node type
node_type = "city"

# Parse the file contents
file_contents = file_contents.split("\n")[3:]
for line in file_contents:
    if line:
        data = line.split("\t")
        # Create a dictionary of the node data
        node_data = {
            "name": data[3],
            "population": int(data[15]),
            "state": data[5],
            "country": data[4]
        }
        # Load the node into the knowledge graph
        load_node(node_type, node_data)
```

--
AiResult: used 4561 token, cost $0.0091, took 26.5 seconds